In [ ]:
#
#
#

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
# data/reloading_data - CATRIDGES.csv
cartridges = pd.read_csv('data/reloading_data - CATRIDGES.csv')
print(cartridges)

cartridges_to_process = cartridges.loc[:, ['title', 'volume', 'diameter']].to_numpy()
print(cartridges_to_process)

In [ ]:
def process_data_file(c, v, a):
    filename = f'data/reloading_data - %s.csv' % c
    if not os.path.exists(filename):
        return None
    d = pd.read_csv(filename)
    d['cartridge_caliber'] = c
    d['cartridge_volume'] = v
    d['cartridge_diameter'] = a
    return d

frames = [ process_data_file(c, v, a) for c, v, a in cartridges_to_process ]
data = pd.concat(frames, ignore_index=True)

print(data.head())  # DEBUG

In [ ]:
data.shape

In [ ]:
# reloading_data - powder burning rate chart
powder_rates = pd.read_csv('data/reloading_data - powder burning rate chart.csv')
print(powder_rates.head())

#
# verify the data
#
unique_powder_names = powder_rates.powder.unique()
print(len(unique_powder_names), len(powder_rates))
assert np.all(unique_powder_names == powder_rates.powder), "powder_rates is not unique"

In [ ]:
# add a "bullet type" column

orig_columns = [
    'powder', 'start', 'velocity', 'never exceed', 'velocity max', 'press', 'units', 'min oal',
    'cartridge_caliber', 'cartridge_volume', 'cartridge_diameter'
    ]
new_columns  = [
    'bul_weight', 'bul_type', 
    'powder', 's_load', 's_velocity', 'ne_load', 'ne_velocity', 'press', 'units', 'oal',
    'cartridge_caliber', 'cartridge_volume', 'cartridge_diameter'
    ]

lee = pd.DataFrame(columns = new_columns)

bul_weight = None
bul_type = None
for pos in range(0, data.shape[0]):
    line = tuple(data.loc[pos, orig_columns].array)
    # print(123, line) # DEBUG
    if ' grain ' in line[0]: # this is a bullettype "115 grain jacketed"
        bul_weight, bul_type = line[0].split(' grain ')
    else:
        line = (int(bul_weight), bul_type, *line)
        lee.loc[len(lee.index)] = line

In [ ]:
lee.shape

In [ ]:
def powder_id(x):
    try:
        return np.where(powder_rates.powder == x)[0][0]
    except :
        print (x)

def bul_type_id(x):
    return np.where(lee.bul_type.unique() == x)[0][0]


In [ ]:
lee['powder_id'] = [ powder_id(x) for x in lee.powder ]
lee['bul_type_id'] = [ bul_type_id(x) for x in lee.bul_type ]
lee['powder_rate'] = list(powder_rates.rate[lee.powder_id])

In [ ]:
print(lee)
lee.to_csv('lee.csv')
print("saved...")

In [ ]:
for p in ['HP38', 'H110']:
    arr = lee[lee.powder == p][['powder', 'powder_id', 'bul_type_id', 'powder_rate', 'cartridge_volume', 'cartridge_diameter']].to_numpy()
    print(arr[0])
    
print(lee[lee.powder == 'H110'][['powder', 'powder_id', 'bul_type_id', 'powder_rate', 'cartridge_volume', 'cartridge_diameter']])
print('-------------')
print(lee[lee.powder == 'H110'])

In [ ]:
print(powder_rates[powder_rates.powder == 'HP38'][['rate']])

In [ ]:
lee_clean = lee.dropna()

In [ ]:
msk = np.random.rand(len(lee_clean)) < 0.8
train = lee_clean[msk]
test = lee_clean[~msk]

train.describe()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

y_keys = ['s_load', 'ne_load']
x_keys = ['powder_id', 'bul_type_id', 'powder_rate', 'cartridge_volume', 'cartridge_diameter' ]

train_x = np.asanyarray(train[x_keys])
train_y = np.asanyarray(train[y_keys])

test_x = np.asanyarray(test[x_keys])
test_y = np.asanyarray(test[y_keys])
print(train_x.shape)

poly = PolynomialFeatures(degree=3)
train_x_poly = poly.fit_transform(train_x)
train_x_poly.shape

In [ ]:
clf = linear_model.LinearRegression()
train_y_ = clf.fit(train_x_poly, train_y)
# The coefficients
if False:
    print ('Coefficients: ', clf.coef_)
    print ('Intercept: ',clf.intercept_)
    
from sklearn.metrics import r2_score

test_x_poly = poly.fit_transform(test_x)
test_y_ = clf.predict(test_x_poly)

print("Mean absolute error: %.2f" % np.mean(np.absolute(test_y_ - test_y)))
print("Residual sum of squares (MSE): %.2f" % np.mean((test_y_ - test_y) ** 2))
print("R2-score: %.2f" % r2_score(test_y_ , test_y) )

In [ ]:
print(test_x)
print(test_y_)

In [ ]:
# Let's do actual calculations
pwd, rate = lee[lee.powder == 'HP38'][['powder_id', 'powder_rate']].to_numpy()[0]
print(pwd, rate)

actual_x = np.array([[pwd, 0, rate, 0.74, 0.356], 
                     [pwd, 1, rate, 0.74, 0.356], 
                     [pwd, 2, rate, 0.74, 0.356], 
                     [pwd, 3, rate, 0.74, 0.356]])
print('X', actual_x)
actual_x_poly = poly.fit_transform(actual_x)
actual_y_ = clf.predict(actual_x_poly)
print('Y', actual_y_)

In [ ]:
# Let's do actual calculations
pwd, rate = lee[lee.powder == 'v-3N37'][['powder_id', 'powder_rate']].to_numpy()[0]
print(pwd, rate)


# x_keys = ['powder_id', 'bul_type_id', 'powder_rate', 'cartridge_volume', 'cartridge_diameter' ]
actual_x = np.array([[pwd, 0, rate, 0.74, 0.356], 
                     [pwd, 1, rate, 0.74, 0.356], 
                     [pwd, 2, rate, 0.74, 0.356], 
                     [pwd, 3, rate, 0.74, 0.356],
                     [pwd, 4, rate, 0.74, 0.356],
                     [pwd, 5, rate, 0.74, 0.356],
                     [pwd, 6, rate, 0.74, 0.356],
                    ])
print('X', actual_x)
actual_x_poly = poly.fit_transform(actual_x)
actual_y_ = clf.predict(actual_x_poly)
print('Y', actual_y_)

In [ ]:
lee.bul_type.unique()